In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import pandas as pd
import argparse
import sys,os
import tensorflow as tf
from PIL import Image
from tqdm import tqdm
from urllib.request import urlopen

In [2]:
def load_image(filename):
    #Read in the image_data to be classified."""
    return tf.gfile.FastGFile(filename, 'rb').read()

def load_labels(filename):
    #Read in labels, one label per line."""
    return [line.rstrip() for line in tf.gfile.GFile(filename)]

def load_graph(filename):
    #Unpersists graph from file as default graph."""
    with tf.gfile.FastGFile(filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        tf.import_graph_def(graph_def, name='')
        
def load_pb(path_to_pb):
    with tf.gfile.GFile(path_to_pb, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def, name='')
        return graph

In [4]:
graph_path = '/Users/Alex/Documents/Cover/car_classifier/image_classifier/model_mobilenet_v1/output_graph.pb'
load_graph(graph_path)

label_path = '/Users/Alex/Documents/Cover/car_classifier/image_classifier/model_mobilenet_v1/cover_labels.txt'
labels = load_labels(label_path)

input_layer = 'DecodeJpeg/contents:0'
output_layer = 'final_result:0'

num_top_predictions = 3

src = os.path.join('/Volumes/alex_encrypted/model_v1/','test')

In [10]:
## Print input and output names from model
gf = tf.GraphDef()   
m_file = open(graph_path,'rb')
gf.ParseFromString(m_file.read())

with open('somefile.txt', 'a') as the_file:
    for n in gf.node:
        the_file.write(n.name+'\n')

file = open('somefile.txt','r')
data = file.readlines()
print ("output name = ", data[len(data)-1])
file.seek ( 0 )
print ("Input name = ", file.readline())

output name = 
final_result

Input name = 
DecodeJpeg/contents



In [12]:
#tensorboard --logdir=path/to/log-directory

import tensorflow as tf

tflite_convert \
--output_file '/Users/Alex/Documents/Cover/car_classifier/image_classifier/model_v1/model_v1_lite.tflite' \
--graph_def_file '/Users/Alex/Documents/Cover/car_classifier/image_classifier/model_v1/output_graph.pb' \
--input_arrays 'DecodeJpeg/contents' \
--output_arrays 'final_result'  

UsageError: Line magic function `%tensorboard` not found.


In [ ]:
# Run just one image
# one_image_url = ''
one_image_filepath = '/Users/Alex/Desktop/odo1.png'

with tf.Session() as sess:
    image_data=load_image(one_image_filepath)
    softmax_tensor=sess.graph.get_tensor_by_name(output_layer)
    predictions,=sess.run(softmax_tensor, {input_layer: image_data})

    # Sort to show labels in order of confidence             
    top_k = predictions.argsort()[-num_top_predictions:][::-1]
    predict_txt = ''
    i = 0
    for node_id in top_k:
        # get top predictions only
        if i == 0:
            top_prediction = labels[node_id]
            top_score = str(round(predictions[node_id], 5))
        # get all predictions
        predicted_label = labels[node_id]
        score = predictions[node_id]
        predict_txt += predicted_label + ': ' + str(round(score, 5)) + ', '

        i += 1
        #print(f, predicted_label, score)

    new_line = [one_image_filepath, 'top_prediction: ' + top_prediction, top_score, predict_txt]
    print(new_line)
    display(Image.open(one_image_filepath))

In [5]:
final_list = [['image_name', 'image_path', 'actual_label', 'top_prediction', 'guess_correct', 'top_score', 'all_predictions' ]]


def run_graph(src, labels, input_layer_name, output_layer_name, num_top_predictions):
    
    with tf.Session() as sess:
        for label in labels:
            label = label.replace(' ', '_')
            # Feed the image_data as input to the graph.
            for f in os.listdir(os.path.join(src, label)):
                if f.endswith('.jpg'):
                    image_data=load_image(os.path.join(src, label, f))
                    softmax_tensor=sess.graph.get_tensor_by_name(output_layer_name)
                    predictions,=sess.run(softmax_tensor, {input_layer_name: image_data})

                    # Sort to show labels in order of confidence             
                    top_k = predictions.argsort()[-num_top_predictions:][::-1]
                    predict_txt = ''
                    i = 0
                    for node_id in top_k:
                        # get top predictions only
                        if i == 0:
                            top_prediction = labels[node_id]
                            top_score = str(round(predictions[node_id], 5))
                        # get all predictions
                        predicted_label = labels[node_id]
                        score = predictions[node_id]
                        predict_txt += predicted_label + ': ' + str(round(score, 5)) + ', '

                        i += 1
                        #print(f, predicted_label, score)

                    new_line = [f, os.path.join(src, label), label, top_prediction, label == top_prediction.replace(' ', '_'), top_score, predict_txt]
                    #print(new_line)
                    final_list.append(new_line)

In [6]:
run_graph(src, labels, input_layer, output_layer, num_top_predictions)

In [7]:
out_file_path = '/Users/Alex/Documents/Cover/car_classifier/image_classifier/mobilenet_results.csv'

df = pd.DataFrame(final_list)
df.to_csv(out_file_path, index=False, header= False)